In [1]:
isinstance([], list)

True

In [5]:
isinstance((), object)

True

## 虚拟子类

In [6]:
import abc


class AbstractDict(metaclass=abc.ABCMeta):
    def foo(self):
        return None

AbstractDict.register(dict)

dict

In [8]:
isinstance({}, AbstractDict)

True

In [10]:
issubclass(dict, AbstractDict)

True

In [12]:
class MySequence(metaclass=abc.ABCMeta):
    pass

MySequence.register(list)
MySequence.register(tuple)

tuple

In [14]:
isinstance([], MySequence)

True

In [15]:
@MySequence.register
class Mycustlink(object):
    pass


isinstance(Mycustlink(), MySequence)

True

In [16]:
class MyDuck(metaclass=abc.ABCMeta):
    @classmethod
    def __subclasshook__(cls, subclass):
        quack = getattr(subclass, 'quack', None)
        return callable(quack)


class Duck(object):
    def quack(self):
        pass


class NotDuck(object):
    quack = "foo"

issubclass(Duck, MyDuck)

True

In [17]:
issubclass(NotDuck, MyDuck)

False

In [19]:
class AbstractDuck(metaclass=abc.ABCMeta):
    @classmethod
    def __subclasshook__(cls, subclass):
        quack = getattr(subclass, 'quack', None)
        if callable(quack):
            return True
        return NotImplemented

In [20]:
issubclass(NotDuck, AbstractDuck)

False

In [22]:
AbstractDuck.register(NotDuck)
issubclass(NotDuck, AbstractDuck)

True

## 声明协议

In [33]:
from datetime import datetime as dt


class Task(object):
    def __init__(self):
        self.runs = []

    def run(self):
        start = dt.now()
        result = self._run()
        self.runs.append({
            'start': start,
            'end': dt.now(),
            'result': result
        })
        return result
    
    def _run(self):
        raise NotImplementedError

In [34]:
t = Task()

In [35]:
t.run()

NotImplementedError: 

In [37]:
class Taskclass(Task):

    def __init__(self):
        pass

In [38]:
Taskclass()

In [44]:
class TaskMeta(type):
    def __new__(cls, name, bases, attrs, *args, **kwargs):
        if attrs.pop('abstract', False):
            return super(TaskMeta,cls).__new__(cls, name, bases, attrs, *args, **kwargs)
        
        new_class = super(TaskMeta,cls).__new__(cls, name, bases, attrs)
        
        if not hasattr(new_class, '_run') or not callable(new_class._run):
            raise TypeError
        return new_class
    
    
class Task(metaclass=TaskMeta):
    abstract = True
    
    def __init__(self):
        self.runs = []
    
    def run(self):
        start = dt.now()
        result = self._run()
        self.runs.append({
            'start': start,
            'end': dt.now(),
            'result': result
        })
        return result

In [45]:
t = Task()

In [46]:
t.run()

AttributeError: 'Task' object has no attribute '_run'

In [48]:
class Tasksub(Task):
    pass

TypeError: 

### 抽象基类

In [39]:
class Task(metaclass=abc.ABCMeta):
    def __init__(self):
        self.runs = []
    
    def run(self):
        start = dt.now()
        result = self._run()
        self.runs.append({
            'start': start,
            'end': dt.now(),
            'result': result
        })
        return result
    
    @abc.abstractmethod
    def _run(self):
        pass

In [40]:
t = Task()

TypeError: Can't instantiate abstract class Task with abstract methods _run

In [42]:
class SubTask(Task):
    pass

In [49]:
class OtherSubTask(Task):
    def _run(self):
        return 2+2
    
ost = OtherSubTask()
ost.run()

4

In [50]:
ost.runs

[{'end': datetime.datetime(2018, 12, 10, 14, 47, 3, 297357),
  'result': 4,
  'start': datetime.datetime(2018, 12, 10, 14, 47, 3, 297357)}]

### 抽象属性

In [54]:
class AbstractClass(metaclass=abc.ABCMeta):
    @property
    @abc.abstractmethod
    def foo(self):
        pass
    
    
class InvalidChild(AbstractClass):
    pass

In [55]:
ic = InvalidChild()

TypeError: Can't instantiate abstract class InvalidChild with abstract methods foo

In [56]:
class ValidChild(AbstractClass):
    @property
    def foo(self):
        return 'foo'

In [57]:
ic = ValidChild()

In [58]:
ic.foo

'foo'

### 抽象类或静态方法

In [77]:
class AbstractClass(metaclass=abc.ABCMeta):
    @classmethod
    @abc.abstractmethod
    def foo(cls):
        return 42

In [78]:
AbstractClass.foo()

42

In [84]:
class InValidChild(AbstractClass):
    pass
InvalidChild.foo()

45

In [81]:
class ValidChild(AbstractClass):
    @classmethod
    def foo(cls):
        return 45

    def __init__(self):
        pass

    pass

ic = ValidChild()

In [82]:
ValidChild.foo()

45

In [83]:
ic = ValidChild()
ic.foo()

45

In [85]:
from collections.abc import Sized


class Foo(object):
    def __len__(self):
        return 42
    
issubclass(Foo, Sized)

True

In [86]:
class Bar(Sized):
    def __len__(self):
        return 42
    
    def __init__(self):
        pass

b = Bar()

In [87]:
len(b)

42

In [88]:
isinstance(b, Sized)

True

In [1]:
from collections.abc import MutableSequence

isinstance([], MutableSequence)

True